In [18]:
import pickle
import pandas as pd
import boto3
import os
import numpy as np
from tqdm import tqdm

from deepmatch.layers import custom_objects

In [17]:
def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))

s3client = boto3.client('s3')
bucket = 'sagemaker-us-east-1-002224604296'
local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# 行为数据加载
file_name_list = ['ub_item_embeddings.npy']
s3_folder = 'recommender-system-film-mk/1/feature/action/'
sync_s3(file_name_list, s3_folder, local_folder)
# youtubednn模型相关文件
file_name_list = ['user_embeddings.h5']
s3_folder = 'recommender-system-film-mk/1/model/recall/youtubednn/'
sync_s3(file_name_list, s3_folder, local_folder)
# youtubednn模型相关文件
file_name_list = ['raw_embed_user_mapping.pickle', 'raw_embed_item_mapping.pickle']
s3_folder = 'recommender-system-film-mk/1/feature/action/'
sync_s3(file_name_list, s3_folder, local_folder)

file preparation: download src key recommender-system-film-mk/1/feature/action/ub_item_embeddings.npy to dst key info/ub_item_embeddings.npy
file preparation: download src key recommender-system-film-mk/1/model/recall/youtubednn/user_embeddings.h5 to dst key info/user_embeddings.h5
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_user_mapping.pickle to dst key info/raw_embed_user_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_item_mapping.pickle to dst key info/raw_embed_item_mapping.pickle


In [49]:
# 加载pickle文件
file_to_load = open("../recall-batch/info/recall_batch_result.pickle", "rb")
recall_batch_result = pickle.load(file_to_load)
file_to_load = open("../recall-batch/info/recall_batch_result.pickle", "rb")
recall_batch_result = pickle.load(file_to_load)
file_to_load = open("../recall-batch/info/movie_id_movie_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)
file_to_load = open("../recall-batch/info/portrait.pickle", "rb")
user_portrait = pickle.load(file_to_load)
file_to_load = open("../recall-batch/info/raw_embed_user_mapping.pickle", "rb")
raw_embed_user_mapping = pickle.load(file_to_load)
file_to_load = open("../recall-batch/info/raw_embed_item_mapping.pickle", "rb")
raw_embed_item_mapping = pickle.load(file_to_load)
ub_item_embeddings = np.load("info/ub_item_embeddings.npy")
# 加载模型
user_embedding_model = load_model('info/user_embeddings.h5', custom_objects)
file_to_load = open("../recall-batch/info/movie_id_movie_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)
print("length of movie_id v.s. movie_property {}".format(len(dict_id_content)))
# 加载pickle文件
file_to_load = open("info/raw_embed_item_mapping.pickle", "rb")
dict_item_mapping = pickle.load(file_to_load)
file_to_load = open("info/raw_embed_user_mapping.pickle", "rb")
dict_user_mapping = pickle.load(file_to_load)

# 加载所有人的数据
action_data_pddf = pd.read_csv("../recall-batch/info/action.csv", sep='\t')
print("load {} action data".format(len(action_data_pddf)))
action_data_pddf.head()

length of movie_id v.s. movie_property 33767
load 207341 action data


,label,userid,programId,programType,action,timeStamp,title,genres
0,1,13396,1690115,1,2,1608198657,jungleland,drama
1,1,13396,1690115,1,2,1608198696,jungleland,drama
2,1,170811011119,1690115,1,2,1610225454,jungleland,drama
3,1,170811012694,1690115,1,2,1612996593,jungleland,drama
4,1,24384,1690115,1,2,1614031172,jungleland,drama


In [75]:
sample_data_pddf = action_data_pddf[action_data_pddf['programType']==1][['label','userid','programId','timeStamp']]
sample_data_pddf.head()

,label,userid,programId,timeStamp
0,1,13396,1690115,1608198657
1,1,13396,1690115,1608198696
2,1,170811011119,1690115,1610225454
3,1,170811012694,1690115,1612996593
4,1,24384,1690115,1614031172


In [195]:
sample_data_pddf.sort_values('timeStamp',inplace=True)

sample_data_pddf_click = sample_data_pddf[sample_data_pddf['label']==1]
user_click_record = {}
for reviewerID, hist in tqdm(sample_data_pddf_click.groupby('userid')):
    current_user_time_list = {}
    pos_list = hist['programId'].tolist()
    time_list = hist['timeStamp'].tolist()
    for idx, t in enumerate(time_list):
        current_user_time_list[t] = pos_list[idx:]
    user_click_record[str(reviewerID)] = current_user_time_list
sample_data_pddf.head()

100%|██████████| 1802/1802 [00:00<00:00, 4852.58it/s]


,label,userid,programId,timeStamp,userid_feat,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user_feature_4,...,item_feature_28,item_feature_29,item_feature_30,item_feature_31,sparse_feature_0,sparse_feature_1,sparse_feature_2,sparse_feature_3,sparse_feature_4,sparse_feature_5
177838,0,13396,1690098,1608198651,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,-0.001560,0.002155,0.002172,-0.002128,documentary,liz garbus,stacey abrams|debo adegbile|jayla allen|carol ...,english,pg-13,2020
10,1,13396,1690115,1608198651,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,0.000084,0.000009,-0.000231,-0.000111,drama,max winkler,jack oconnell|charlie hunnam|jessica barden|jo...,english,r,2019
1612,0,13396,1690113,1608198653,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,-0.000929,0.001947,0.001885,-0.001856,horror,justin dix,robert taylor|alyssa sutherland|nathan phillip...,english,ma15+,2019
112324,0,13396,1690097,1608198653,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,-0.000063,-0.000026,0.000264,0.000080,family,charles martin smith,luke treadaway|kristina tonteri-young|phaldut ...,english,12,2020
0,1,13396,1690115,1608198657,"[-0.023948096, 0.020576214, 0.03755351, 0.0155...",-0.023948,0.020576,0.037554,0.015521,0.018013,...,0.000084,0.000009,-0.000231,-0.000111,drama,max winkler,jack oconnell|charlie hunnam|jessica barden|jo...,english,r,2019


In [77]:
def user_embed(x, user_portrait):    
    if x in user_portrait.keys():
#         print(user_portrait[x])
        return user_portrait[x]['ub_embeddding'][0]
    else:
        return [0]*32
    
def user_id_feat(x, i):
    return x[i]
#     return pd.Series(f_dict)

def user_embed_func(x, user_click_record=user_click_record, user_embedding_model=user_embedding_model, dict_item_mapping=dict_item_mapping, dict_user_mapping=dict_user_mapping):
    current_time_stamp = x['timeStamp']
    if str(x['userid']) in user_click_record.keys():
        current_click_record = user_click_record[str(x['userid'])]
#         last_ts = list(current_click_record.keys())[0]
#         print("current stamp is {}".format(current_time_stamp))
        input_item_list = []
        for ts in current_click_record.keys():
            if current_time_stamp < ts:
#                 print("found last ts {}".format(ts))
                input_item_list = current_click_record[ts]
                break
        # get embedding value
        user_id = x['userid']
        watch_list_len = 50
        map_input_item_list = np.array([[0]*watch_list_len])
        watch_len = len(input_item_list)
        map_user_id = dict_user_mapping[str(user_id)]
        for cnt, item in enumerate(input_item_list):
            if cnt < 50:
                map_input_item_list[0][cnt] = dict_item_mapping[str(item)]
        model_input = {}
        model_input['user_id'] = np.array([int(map_user_id)])
        model_input['hist_movie_id'] = map_input_item_list
        model_input['hist_len'] = np.array([watch_len])

        # 更新用户的embeddings
    #     print("model input {}".format(model_input))
        updated_user_embs = user_embedding_model.predict(
            model_input, batch_size=2 ** 12)
        return updated_user_embs[0]
    else:
#         print("zero click!!")
        return [0]*32
    
def item_embed(x, raw_embed_item_mapping, ub_item_embeddings):
    embed_item_idx = raw_embed_item_mapping[str(x)]
    if  int(embed_item_idx) < len(ub_item_embeddings):
#         print(user_portrait[x])
        return ub_item_embeddings[int(embed_item_idx)]
    else:
        return [0]*32
    
def item_id_feat(x, i):
    return x[i]
#     return pd.Series(f_dict)

def sparse_item_id_feat(x, mt, dict_id_content=dict_id_content):
    result = dict_id_content[str(x)][mt]
    if result[0] is None:
        return None
    else:
        return '|'.join(result)

In [196]:
# user id feature - user embedding
sample_data_pddf['userid_feat'] = sample_data_pddf.apply(user_embed_func, axis=1)
for i in range(32):
    sample_data_pddf['user_feature_{}'.format(i)] = sample_data_pddf['userid_feat'].apply(lambda x: user_id_feat(x, i))
# item id feature - item embedding
sample_data_pddf['itemid_feat'] = sample_data_pddf['programId'].apply(lambda x: item_embed(x, raw_embed_item_mapping, ub_item_embeddings))
for i in range(32):
    sample_data_pddf['item_feature_{}'.format(i)] = sample_data_pddf['itemid_feat'].apply(lambda x: item_id_feat(x, i))
# sparse feature
popularity_method_list = ['category', 'director',
                          'actor', 'language', 'level', 'year']
feature_num = 6
for i, mt in enumerate(popularity_method_list):
    sample_data_pddf['sparse_feature_{}'.format(i)] = sample_data_pddf['programId'].apply(lambda x: sparse_item_id_feat(x, mt))

In [197]:
sample_data_pddf.head()

,label,userid,programId,timeStamp,userid_feat,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user_feature_4,...,item_feature_28,item_feature_29,item_feature_30,item_feature_31,sparse_feature_0,sparse_feature_1,sparse_feature_2,sparse_feature_3,sparse_feature_4,sparse_feature_5
177838,0,13396,1690098,1608198651,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,-0.001560,0.002155,0.002172,-0.002128,documentary,liz garbus,stacey abrams|debo adegbile|jayla allen|carol ...,english,pg-13,2020
10,1,13396,1690115,1608198651,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,0.000084,0.000009,-0.000231,-0.000111,drama,max winkler,jack oconnell|charlie hunnam|jessica barden|jo...,english,r,2019
1612,0,13396,1690113,1608198653,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,-0.000929,0.001947,0.001885,-0.001856,horror,justin dix,robert taylor|alyssa sutherland|nathan phillip...,english,ma15+,2019
112324,0,13396,1690097,1608198653,"[-0.023859158, 0.020460255, 0.03747003, 0.0154...",-0.023859,0.020460,0.037470,0.015414,0.018150,...,-0.000063,-0.000026,0.000264,0.000080,family,charles martin smith,luke treadaway|kristina tonteri-young|phaldut ...,english,12,2020
0,1,13396,1690115,1608198657,"[-0.023948096, 0.020576214, 0.03755351, 0.0155...",-0.023948,0.020576,0.037554,0.015521,0.018013,...,0.000084,0.000009,-0.000231,-0.000111,drama,max winkler,jack oconnell|charlie hunnam|jessica barden|jo...,english,r,2019


In [201]:
# experiment with deepfm
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names
from deepctr.estimator.models import DeepFMEstimator

In [147]:
data = pd.read_csv('/home/ec2-user/workplace/DeepCTR/examples/criteo_sample.txt')
sparse_features = ['C' + str(i)for i in range(1, 27)]
dense_features = ['I'+str(i) for i in range(1, 14)]
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)
target = ['label']

In [148]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
nms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = nms.fit_transform(data[dense_features])

In [149]:
data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,...,8,66,0,0,3,0,1,96,0,0
1,0,0.0,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,...,7,52,0,0,47,0,7,112,0,0
2,0,0.0,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,...,8,49,0,0,25,0,6,53,0,0
3,0,0.0,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,...,8,37,0,0,156,0,0,32,0,0
4,0,0.0,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,...,8,14,5,3,9,0,0,5,1,47


In [150]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4) for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1) for feat in dense_features]

In [151]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [152]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model.fit(train_model_input, train[target].values,batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Train on 128 samples, validate on 32 samples
Epoch 1/10
128/128 - 4s - loss: 0.7414 - binary_crossentropy: 0.7414 - val_loss: 0.6839 - val_binary_crossentropy: 0.6839
Epoch 2/10
128/128 - 0s - loss: 0.7229 - binary_crossentropy: 0.7229 - val_loss: 0.6685 - val_binary_crossentropy: 0.6685
Epoch 3/10
128/128 - 0s - loss: 0.7054 - binary_crossentropy: 0.7053 - val_loss: 0.6539 - val_binary_crossentropy: 0.6538
Epoch 4/10
128/128 - 0s - loss: 0.6885 - binary_crossentropy: 0.6885 - val_loss: 0.6397 - val_binary_crossentropy: 0.6396
Epoch 5/10
128/128 - 0s - loss: 0.6719 - binary_crossentropy: 0.6719 - val_loss: 0.6258 - val_binary_crossentropy: 0.6258
Epoch 6/10
128/128 - 0s - loss: 0.6557 - binary_crossentropy: 0.6556 - val_loss: 0.6124 - val_binary_crossentropy: 0.6124
Epoch 7/10
128/128 - 0s - loss: 0.6395 - binary_crossentropy: 0.6395 - val_loss: 0.5993 - val_binary_crossentropy: 0.5993
Epoch 8/10
128/128 - 0s - loss: 0.6235 - binary_crossentropy: 0.6235 - val_loss: 0.5864 - val_binary_

In [153]:
pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test LogLoss 0.595
test AUC 0.6211


In [198]:
mk_data = sample_data_pddf[['label','timeStamp']]
dense_feature_size = 64
sparse_feature_size = 6
for i in range(dense_feature_size):
    if i < 32:
        mk_data['I{}'.format(i+1)] = sample_data_pddf['user_feature_{}'.format(i)]
    else:
        mk_data['I{}'.format(i+1)] = sample_data_pddf['item_feature_{}'.format(i-32)]
for i in range(sparse_feature_size):
    mk_data['C{}'.format(i+1)] = sample_data_pddf['sparse_feature_{}'.format(i)]
    
mk_sparse_features = ['C' + str(i)for i in range(1, sparse_feature_size+1)]
mk_dense_features = ['I'+str(i) for i in range(1, dense_feature_size+1)]
mk_data[mk_sparse_features] = mk_data[mk_sparse_features].fillna('-1', )
mk_data[mk_dense_features] = mk_data[mk_dense_features].fillna(0,)

mk_target = ['label']

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [202]:
for feat in mk_sparse_features:
    lbe = LabelEncoder()
    mk_data[feat] = lbe.fit_transform(mk_data[feat])
nms = MinMaxScaler(feature_range=(0,1))
mk_data[mk_dense_features] = nms.fit_transform(mk_data[mk_dense_features])

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is t

In [ ]:
mk_data.head()

In [ ]:
mk_fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4) for i,feat in enumerate(mk_sparse_features)] + [DenseFeat(feat, 1) for feat in mk_dense_features]

In [ ]:
mk_dnn_feature_columns = mk_fixlen_feature_columns
mk_linear_feature_columns = mk_fixlen_feature_columns
mk_feature_names = get_feature_names(mk_linear_feature_columns + mk_dnn_feature_columns)

In [ ]:
mk_train, mk_test = train_test_split(mk_data, test_size=0.2)
mk_train_model_input = {name:mk_train[name].values for name in mk_feature_names}
mk_test_model_input = {name:mk_test[name].values for name in mk_feature_names}
model = DeepFM(mk_linear_feature_columns,mk_dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model.fit(mk_train_model_input, mk_train[mk_target].values,batch_size=256, epochs=100, verbose=2, validation_split=0.2, )

In [ ]:
mk_pred_ans = model.predict(mk_test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(mk_test[target].values, mk_pred_ans), 4))
print("test AUC", round(roc_auc_score(mk_test[target].values, mk_pred_ans), 4))

In [173]:
mk_test

,label,timeStamp,I1,I2,I3,I4,I5,I6,I7,I8,...,I61,I62,I63,I64,C1,C2,C3,C4,C5,C6
166244,0,1613309103,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.384832,0.684664,0.671938,0.298592,153,0,1,12,0,33
132310,0,1612323553,0.159546,0.833294,0.914067,0.790824,0.854998,0.140103,0.881772,0.848952,...,0.263077,0.561791,0.637766,0.419512,55,0,1,12,0,32
2672,0,1612201540,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.311489,0.548374,0.604006,0.447087,113,0,1,4,0,30
1475,0,1611009592,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.357301,0.458754,0.616213,0.477319,38,161,95,4,24,30
201317,0,1611186880,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.302710,0.539023,0.610116,0.451178,113,0,1,4,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166824,0,1614025221,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.384832,0.684664,0.671938,0.298592,153,0,1,12,0,33
491,0,1613972624,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.380862,0.660122,0.819941,0.161234,0,41,125,4,17,20
121146,0,1612369049,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.189502,0.628831,0.720951,0.339529,96,0,1,12,0,32
171185,0,1613949316,0.158695,0.831660,0.914625,0.792079,0.854499,0.141787,0.881496,0.848250,...,0.276488,0.554005,0.626504,0.431435,96,0,1,12,0,32


In [172]:
mk_test_model_input

{'C1': array([153,  55, 113, ...,  96,  96,  55]),
 'C2': array([ 0,  0,  0, ...,  0,  0, 72]),
 'C3': array([ 1,  1,  1, ...,  1,  1, 51]),
 'C4': array([12, 12,  4, ..., 12, 12, 12]),
 'C5': array([ 0,  0,  0, ...,  0,  0, 25]),
 'C6': array([33, 32, 30, ..., 32, 32, 33]),
 'I1': array([1.        , 0.15954628, 1.        , ..., 1.        , 0.15869523,
        0.14500619]),
 'I2': array([0.        , 0.83329359, 0.        , ..., 0.        , 0.8316599 ,
        0.84462579]),
 'I3': array([0.        , 0.91406713, 0.        , ..., 0.        , 0.91462521,
        0.92009508]),
 'I4': array([0.        , 0.79082373, 0.        , ..., 0.        , 0.79207938,
        0.83341005]),
 'I5': array([0.        , 0.85499781, 0.        , ..., 0.        , 0.85449917,
        0.82499527]),
 'I6': array([1.        , 0.14010344, 1.        , ..., 1.        , 0.14178687,
        0.11903088]),
 'I7': array([0.        , 0.88177168, 0.        , ..., 0.        , 0.8814962 ,
        0.89015637]),
 'I8': array([0. 

In [ ]:
model.save_weights('DeepFM_w.h5')

In [ ]:
!tar -czvf model.tar.gz ./DeepFM_w.h5
!aws s3 cp model.tar.gz s3://sagemaker-us-east-1-002224604296/recommender-system-film-mk/1/model/rank/action/deepfm/latest/

In [2]:
# rank batch logic
import pickle
import boto3
import os
import numpy as np
import itertools
import tarfile
import pandas as pd
from tqdm import tqdm
from tensorflow.python.keras.models import Model, save_model, load_model

########################################
# 从s3同步数据
########################################
def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))

s3client = boto3.client('s3')
bucket = 'sagemaker-us-east-1-002224604296'
local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# recall batch 结果记载
file_name_list = ['recall_batch_result.pickle']
s3_folder = 'recommender-system-film-mk/1/feature/recommend-list/movie'
sync_s3(file_name_list, s3_folder, local_folder)
# ub数据加载
file_name_list = ['ub_item_embeddings.npy', 'raw_embed_item_mapping.pickle', 'raw_embed_user_mapping.pickle']
s3_folder = 'recommender-system-film-mk/1/feature/action/'
sync_s3(file_name_list, s3_folder, local_folder)
# 用户画像数据加载
file_name_list = ['portrait.pickle']
s3_folder = 'recommender-system-film-mk/1/feature/recommend-list/portrait'
sync_s3(file_name_list, s3_folder, local_folder)
# 倒排列表的pickle文件
file_name_list = ['movie_id_movie_property_dict.pickle']
s3_folder = 'recommender-system-film-mk/1/feature/content/inverted-list/'
sync_s3(file_name_list, s3_folder, local_folder)
# deepfm模型文件下载
file_name_list = ['model.tar.gz']
s3_folder = 'recommender-system-film-mk/1/model/rank/action/deepfm/latest/'
sync_s3(file_name_list, s3_folder, local_folder)


# 加载pickle文件
file_to_load = open("info/movie_id_movie_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)
print("length of movie_id v.s. movie_property {}".format(len(dict_id_content)))
file_to_load = open("info/recall_batch_result.pickle", "rb")
recall_batch_result = pickle.load(file_to_load)
file_to_load = open("info/recall_batch_result.pickle", "rb")
recall_batch_result = pickle.load(file_to_load)
file_to_load = open("info/movie_id_movie_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)
file_to_load = open("info/portrait.pickle", "rb")
user_portrait = pickle.load(file_to_load)
file_to_load = open("info/raw_embed_user_mapping.pickle", "rb")
raw_embed_user_mapping = pickle.load(file_to_load)
file_to_load = open("info/raw_embed_item_mapping.pickle", "rb")
raw_embed_item_mapping = pickle.load(file_to_load)
ub_item_embeddings = np.load("info/ub_item_embeddings.npy")
# 加载模型
user_embedding_model = load_model('info/user_embeddings.h5', custom_objects)
# 加载pickle文件
file_to_load = open("info/raw_embed_item_mapping.pickle", "rb")
dict_item_mapping = pickle.load(file_to_load)
file_to_load = open("info/raw_embed_user_mapping.pickle", "rb")
dict_user_mapping = pickle.load(file_to_load)
# 解压缩deepfm模型
tar = tarfile.open("info/model.tar.gz","r:gz")
file_names = tar.getnames()
for file_name in file_names:
    tar.extract(file_name, "info/")
tar.close
# 加载recall结果
file_to_load = open("info/recall_batch_result.pickle", "rb")
dict_recall_result = pickle.load(file_to_load)


file preparation: download src key recommender-system-film-mk/1/feature/recommend-list/movie/recall_batch_result.pickle to dst key info/recall_batch_result.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/ub_item_embeddings.npy to dst key info/ub_item_embeddings.npy
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_item_mapping.pickle to dst key info/raw_embed_item_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_user_mapping.pickle to dst key info/raw_embed_user_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/recommend-list/portrait/portrait.pickle to dst key info/portrait.pickle
file preparation: download src key recommender-system-film-mk/1/feature/content/inverted-list/movie_id_movie_property_dict.pickle to dst key info/movie_id_movie_property_dict.pickle
file preparation: download src key recommender-system-film-

NameError: name 'custom_objects' is not defined

In [205]:
data_input_pddf_dict = {}
data_input_pddf_dict['userId'] = []
data_input_pddf_dict['programId'] = []
for user_k,result_v in dict_recall_result.items():
    for item_v in result_v.keys():
        data_input_pddf_dict['userId'].append(str(user_k))
        data_input_pddf_dict['programId'].append(str(item_v))
data_input_pddf = pd.DataFrame.from_dict(data_input_pddf_dict)
data_input_pddf.head()

,userId,programId
0,10541,1688025
1,10541,1688611
2,10541,1680531
3,10541,1688005
4,10541,1680576


In [206]:
def user_embed(x, user_portrait):    
    if x in user_portrait.keys():
#         print(user_portrait[x])
        return user_portrait[x]['ub_embeddding'][0]
    else:
        return [0]*32
    
def user_id_feat(x, i):
    return x[i]
#     return pd.Series(f_dict)

def user_embed_func(x, user_click_record=user_click_record, user_embedding_model=user_embedding_model, dict_item_mapping=dict_item_mapping, dict_user_mapping=dict_user_mapping):
    current_time_stamp = x['timeStamp']
    if str(x['userid']) in user_click_record.keys():
        current_click_record = user_click_record[str(x['userid'])]
#         last_ts = list(current_click_record.keys())[0]
#         print("current stamp is {}".format(current_time_stamp))
        input_item_list = []
        for ts in current_click_record.keys():
            if current_time_stamp < ts:
#                 print("found last ts {}".format(ts))
                input_item_list = current_click_record[ts]
                break
        # get embedding value
        user_id = x['userid']
        watch_list_len = 50
        map_input_item_list = np.array([[0]*watch_list_len])
        watch_len = len(input_item_list)
        map_user_id = dict_user_mapping[str(user_id)]
        for cnt, item in enumerate(input_item_list):
            if cnt < 50:
                map_input_item_list[0][cnt] = dict_item_mapping[str(item)]
        model_input = {}
        model_input['user_id'] = np.array([int(map_user_id)])
        model_input['hist_movie_id'] = map_input_item_list
        model_input['hist_len'] = np.array([watch_len])

        # 更新用户的embeddings
    #     print("model input {}".format(model_input))
        updated_user_embs = user_embedding_model.predict(
            model_input, batch_size=2 ** 12)
        return updated_user_embs[0]
    else:
#         print("zero click!!")
        return [0]*32
    
def item_embed(x, raw_embed_item_mapping, ub_item_embeddings):
    if str(x) not in raw_embed_item_mapping.keys():
        return [0]*32
    embed_item_idx = raw_embed_item_mapping[str(x)]
    if  int(embed_item_idx) < len(ub_item_embeddings):
#         print(user_portrait[x])
        return ub_item_embeddings[int(embed_item_idx)]
    else:
        return [0]*32
    
def item_id_feat(x, i):
    return x[i]
#     return pd.Series(f_dict)

def sparse_item_id_feat(x, mt, dict_id_content=dict_id_content):
    result = dict_id_content[str(x)][mt]
    if result[0] is None:
        return None
    else:
        return '|'.join(result)

In [207]:
# user id feature - user embedding
data_input_pddf['userid_feat'] = data_input_pddf['userId'].apply(lambda x: user_embed(x, user_portrait))
for i in range(32):
    data_input_pddf['user_feature_{}'.format(i)] = data_input_pddf['userid_feat'].apply(lambda x: user_id_feat(x, i))
# item id feature - item embedding
data_input_pddf['itemid_feat'] = data_input_pddf['programId'].apply(lambda x: item_embed(x, raw_embed_item_mapping, ub_item_embeddings))
for i in range(32):
    data_input_pddf['item_feature_{}'.format(i)] = data_input_pddf['itemid_feat'].apply(lambda x: item_id_feat(x, i))
# sparse feature
popularity_method_list = ['category', 'director',
                          'actor', 'language', 'level', 'year']
feature_num = 6
for i, mt in enumerate(popularity_method_list):
    data_input_pddf['sparse_feature_{}'.format(i)] = data_input_pddf['programId'].apply(lambda x: sparse_item_id_feat(x, mt))

In [209]:
mk_test_data = data_input_pddf[['userId','programId']]
dense_feature_size = 64
sparse_feature_size = 6
for i in range(dense_feature_size):
    if i < 32:
        mk_test_data['I{}'.format(i+1)] = data_input_pddf['user_feature_{}'.format(i)]
    else:
        mk_test_data['I{}'.format(i+1)] = data_input_pddf['item_feature_{}'.format(i-32)]
for i in range(sparse_feature_size):
    mk_test_data['C{}'.format(i+1)] = data_input_pddf['sparse_feature_{}'.format(i)]
    
mk_sparse_features = ['C' + str(i)for i in range(1, sparse_feature_size+1)]
mk_dense_features = ['I'+str(i) for i in range(1, dense_feature_size+1)]
mk_test_data[mk_sparse_features] = mk_test_data[mk_sparse_features].fillna('-1', )
mk_test_data[mk_dense_features] = mk_test_data[mk_dense_features].fillna(0,)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [210]:
for feat in mk_sparse_features:
    lbe = LabelEncoder()
    mk_test_data[feat] = lbe.fit_transform(mk_test_data[feat])
nms = MinMaxScaler(feature_range=(0,1))
mk_test_data[mk_dense_features] = nms.fit_transform(mk_test_data[mk_dense_features])

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is t

In [211]:
mk_model_input = {name:mk_test_data[name].values for name in mk_feature_names}
deepfm_model = DeepFM(mk_linear_feature_columns,mk_dnn_feature_columns,task='binary')
deepfm_model.load_weights('DeepFM_w.h5')

In [212]:
mk_pred_ans = deepfm_model.predict(mk_model_input, batch_size=256)

In [219]:
mk_test_data['rank_score'] = [v[0] for v in list(mk_pred_ans)]

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [222]:
rank_result = {}
for reviewerID, hist in tqdm(mk_test_data.groupby('userId')):
    candidate_list = hist['programId'].tolist()
    score_list = hist['rank_score'].tolist()
    id_score_dict = dict(zip(candidate_list, score_list))
    sort_id_score_dict = {k: v for k, v in sorted(id_score_dict.items(), key=lambda item: item[1], reverse=True)}
    rank_result[reviewerID] = sort_id_score_dict

100%|██████████| 1802/1802 [00:00<00:00, 3157.41it/s]


In [223]:
n = 0
for k,v in rank_result.items():
    print("key {} and value {}".format(k,v))
    if n > 10:
        break
    n = n + 1

key 10541 and value {'1559549': 0.0033605992794036865, '1686554': 0.0009735524654388428, '1688343': 0.0007589757442474365, '1471589': 0.00014153122901916504, '1471681': 0.00010946393013000488, '1663274': 7.671117782592773e-05, '1663288': 7.671117782592773e-05, '1639594': 2.771615982055664e-06, '1689997': 8.940696716308594e-08, '1690012': 8.940696716308594e-08, '1471801': 2.9802322387695312e-08, '1688025': 0.0, '1688611': 0.0, '1680531': 0.0, '1688005': 0.0, '1680576': 0.0, '1606221': 0.0, '1691146': 0.0, '1605922': 0.0, '1691166': 0.0, '1639610': 0.0, '1606237': 0.0, '1631811': 0.0, '1606239': 0.0, '1631839': 0.0, '1606242': 0.0, '1650030': 0.0, '1605943': 0.0, '1650021': 0.0, '1605940': 0.0, '1690904': 0.0, '1605938': 0.0, '1690872': 0.0, '1690130': 0.0, '1562880': 0.0, '1684910': 0.0, '1690110': 0.0, '1562794': 0.0, '1682423': 0.0, '1690150': 0.0, '1639580': 0.0, '1639640': 0.0, '1688528': 0.0, '1471796': 0.0, '1686555': 0.0, '1686553': 0.0}
key 10899 and value {'1691146': 0.0, '1691

In [218]:
[v[0] for v in list(mk_pred_ans)]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 7.671118e-05,
 0.0,
 0.0,
 7.671118e-05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.940697e-08,
 0.0,
 0.0,
 8.940697e-08,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0033605993,
 0.00014153123,
 2.771616e-06,
 2.9802322e-08,
 0.0,
 0.0,
 0.00075897574,
 0.0,
 0.00010946393,
 0.0,
 0.00097355247,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.940697e-08,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0669231e-05,
 0.0,
 1.1026859e-06,
 1.1920929e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.1920929e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.1129